In [1]:
import warnings 
warnings.filterwarnings('ignore')

In [2]:
import cv2

# Load the pre-trained Yunet model
model_path = "face_detection_yunet_2023mar.onnx"
yunet = cv2.FaceDetectorYN_create(model_path, "", (320, 320))

# Set input size for the Yunet model
input_size = (320, 320)  # Yunet works best with this size
yunet.setInputSize(input_size)

# Open the webcam
cap = cv2.VideoCapture(0)  # Change if using an external webcam
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Get original frame size
    orig_h, orig_w = frame.shape[:2]

    # Resize frame to Yunet's expected input size
    frame_resized = cv2.resize(frame, input_size)

    # Detect faces
    faces = yunet.detect(frame_resized)[1]  # Yunet returns (retval, faces), we need only faces

    # Draw detected faces on the original frame (scaling bounding boxes)
    if faces is not None:
        for face in faces:
            # Unpack face data (x, y, width, height, confidence)
            x, y, w, h, confidence = face[:5]

            # Scale bounding box back to original frame size
            x = int(x * orig_w / input_size[0])
            y = int(y * orig_h / input_size[1])
            w = int(w * orig_w / input_size[0])
            h = int(h * orig_h / input_size[1])

            # Draw rectangle around the detected face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Add confidence score as text
            cv2.putText(frame, f"{confidence:.2f}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the output frame
    cv2.imshow("Face Detection - Yunet", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()
